In [1]:
import numpy as np
import simkit as sk
import igl
from simkit.filesystem import get_data_directory

In [ ]:
# -- 2D Meshes -- #
filename = '2d/beam/beam.obj'
filename = '2d/bingby/bingby.obj'
filename = '2d/cthulu/cthulu.obj'

[V, _, _, T, _, _]= igl.readOBJ(get_data_directory() + filename)
V = V[:, :2]  # Keep only the first two dimensions for 2D meshes
print('Vertices:', V.shape)
print('Triangles:', T.shape)

In [ ]:
k= 5
mode = 'elem'
mu = np.ones((T.shape[0],1))
data = sk.heat_clustering_precomp(V, T, mu=mu, k=k, mode=mode)
parts, C, dist, clusters, centroids = sk.heat_clustering_solve(data, max_iters=100)


In [ ]:
import polyscope as ps
import polyscope.imgui as psim

if T.shape[1] == 3:
    elem_type = "faces"
else:
    elem_type = "cells"
if mode == 'elem':
    part_type = elem_type
else:
    part_type = "vertices"

def visualize_heat_clustering(V, T, C, clusters, centroids, dist, parts, mu=None):
    ps.init()

    # Register mesh
    if T.shape[1] == 3:
        msh = ps.register_surface_mesh("mesh", V, T)
    else:
        msh = ps.register_volume_mesh("mesh", V, T)

    # Register centroids and initial cluster labeling
    pnts = ps.register_point_cloud("centroids", C)
    msh.add_scalar_quantity("clusters", parts, enabled=True, cmap="rainbow", defined_on=part_type)

    # Optional stiffness visualization
    if mu is not None:
        msh.add_scalar_quantity("mu", mu.flatten(), defined_on=elem_type, cmap="rainbow")

    # Set up interactive sliders for cluster index and iteration
    current_k = 0
    current_iter = 0

    def callback_func():
        nonlocal current_k, current_iter

        # Distance field for cluster k
        changed, k = psim.SliderInt("k", current_k, v_min=0, v_max=dist.shape[1]-1)
        if changed:
            current_k = k
            msh.add_scalar_quantity("d", dist[:, k], defined_on=part_type, enabled=True, cmap="rainbow")

        # Cluster labeling at iteration
        changed, iter = psim.SliderInt("iter", current_iter, v_min=0, v_max=len(clusters)-1)
        if changed:
            current_iter = iter
            pnts = ps.register_point_cloud("centroids", centroids[iter])
            pnts.update_point_positions(centroids[iter])
            msh.add_scalar_quantity("clusters", clusters[iter], enabled=True, cmap="rainbow", defined_on=part_type)

    ps.set_user_callback(callback_func)
    ps.show()
    ps.remove_all_structures()

visualize_heat_clustering(V, T, C, clusters, centroids, dist, parts, mu=mu)

In [ ]:
# Next version try modifying the stiffness so that there that the top part is stiffer.
BC = igl.barycenter(V, T)

min_y = np.min(BC[:, 1])
max_y = np.max(BC[:, 1])
min_x = np.min(BC[:, 0])
max_x = np.max(BC[:, 0])
y_range = max_y - min_y
x_range = max_x - min_x

stiffness = np.ones((T.shape[0], 1))
top_mask = (BC[:, 1] > 0.5 * y_range + min_y)
middle_mask = (BC[:, 0] > 0.4 * x_range + min_x) & (BC[:, 0] < 0.6 * x_range + min_x)
mask = top_mask & middle_mask
stiffness[mask] = 100.0
k = 8
data = sk.heat_clustering_precomp(V, T, mu=stiffness, k=k, mode=mode)
parts, C, dist, clusters, centroids = sk.heat_clustering_solve(data)
visualize_heat_clustering(V, T, C, clusters, centroids, dist, parts, mu=stiffness)